In [ ]:
##### %matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(16,4))
rc('font',size=12)
rc('text',usetex=False)
rc('image', cmap='viridis')

from qubicpack import qubicpack as qp
from qubicpack.utilities import Qubic_DataDir
import qubic.fibtools as ft
import qubic.plotters as p
import qubic.lin_lib as ll
import qubic.demodulation_lib as dl
import qubic.io
import qubic.sb_fitting as sbfit

from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import pickle
import scipy.signal as scsig
from scipy import interpolate
import os
import healpy as hp
import pandas as pd


In [ ]:
#Set paths
machine = os.uname()[1]
def check_dir(dirfile):
    if os.path.isdir( dirfile): 
        print('dirfile:', dirfile); 
    else: 
        raise ValueError('dirfile {} is not an existing directory'.format(dirfile) )
        
if (machine=='apcdhcp52.in2p3.fr') or (machine=='MacJCH-2.local'):
    difiles = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
    check_dir(dirfiles)
elif machine == 'shaka':
    dirfiles = "/home/mgamboa/qubic/QUBIC/qubiccalibration/Data/150GHz-2019-04-06/"
    check_dir(dirfiles)
elif machine == 'hain':
    dirfiles = "/home/martin/QUBIC/qubiccalibration/Data/Calib-TD/150GHz-2019-04-06/"
    check_dir(dirfiles)

else:
    raise ValueError("*dirfile variable not defined*. You are in machine named: {}. \
    You have to specify a directory where the synthesized beam maps are saved in \
    Flat or Healpix format (directories)".format(machine))

In [ ]:
# Get the data
freq_source = int(dirfiles.split("GHz")[0][-3:]) #130 

# dirfreq = '150GHz-2019-04-06/'
# dirallfreqs = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
# dirfiles = dirallfreqs + dirfreq

c50 = np.cos(np.radians(50))
azmin = -15./c50
azmax = 15./c50


TESNum = 93
# reload(sbfit)
flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, azmin = azmin, azmax=azmax)
npix = len(np.ravel(flatmap))

subplot(1,2,1)
imshow(flatmap,
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], aspect='equal',
      vmin=-3000, vmax=3000)
# colorbar()


In [ ]:
alldata_renorm = np.zeros((256, npix))
for i in range(256):
    flatmap, az, el = sbfit.get_flatmap(i+1, dirfiles, azmin=azmin, azmax=azmax)
    alldata_renorm[i, :] = np.ravel((flatmap - np.mean(flatmap)) / np.std(flatmap))  

In [ ]:
##### Estimating the median of all SB images - to be removed - Origin still unexplained...
med = np.reshape(np.median(alldata_renorm,axis=0),np.shape(flatmap))
mm = np.mean(med)
ss = np.std(med)

imshow(np.reshape(med, np.shape(flatmap)),
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
       aspect='equal')
# colorbar()
title('Median over all S.B. images')

In [ ]:
# reload(sbfit)
TESNum = 93
flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, remove=med, azmin=azmin, azmax=azmax)

mm, ss = ft.meancut(flatmap, 3)
subplot(1,2,1)
imshow(flatmap,
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
       aspect='equal',
       vmin=mm-ss, 
       vmax=mm+10*ss)
# colorbar()
title('TES #{}'.format(TESNum))

In [ ]:
# reload(sbfit)
for TESNum in range(1, 257):
    print(TESNum)
    flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, remove=med, azmin=azmin, azmax=azmax)
    
    #### Instantiate a model
    sbfitmodel3 = sbfit.SbModelIndepPeaks(nrings=2, common_fwhm=True, no_xy_shift=False, distortion=False)
    
    fit, xypeaks = sbfit.fit_sb(flatmap, az, el, sbfitmodel3, verbose=False, resample=False, 
                            doplot=False, extra_title='TES #{}'.format(TESNum))
    
    thefile = open(dirfiles + '/FitSB/fit-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/fit-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(fit[1], thefile)
    thefile.close
    
    thefile = open(dirfiles + '/FitSB/fiterr-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/fiterr-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(fit[2], thefile)
    thefile.close
    
    thefile = open(dirfiles +'/FitSB/xypeaks-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/xypeaks-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(xypeaks, thefile)
    thefile.close


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
# reload(sbfit)
nperpage = 7
nsiglo = 1.
nsighi = 3.
TESmax = 256
fitted_directory = dirfiles +'/FitSB/'
# fitted_directory = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/'

with PdfPages(dirfiles + '/allFitSB_{}.pdf'.format(freq_source)) as pdf:
# with PdfPages('/Users/hamilton/Google Drive/QUBIC/Calib-TD/allFitSB.pdf') as pdf:
    for TESNum in range(1, TESmax+1):
        ipage = (TESNum-1) % nperpage
        print(TESNum, ipage)
        if ipage  == 0:
            rc('figure', figsize=(20,28))
        
        ### Read map
        flatmap, az, el, themap, newxxyy = sbfit.get_flatmap(TESNum, dirfiles, 
                                                             remove=med, azmin=azmin, azmax=azmax,
                                                             fitted_directory = fitted_directory)
        mm, ss = ft.meancut(flatmap-themap,3)
        
        subplot(nperpage, 4, 1+ipage*4)
        imshow(flatmap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=np.min(flatmap)/10, 
               vmax=np.max(flatmap)/2)
        colorbar()
        title('Input Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 2+ipage*4)
        imshow(flatmap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        sh = newxxyy.shape
        for i in range(sh[1]):
            ax = plot(newxxyy[0,i], newxxyy[1,i], 'r.')

        colorbar()
        title('Input Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 3+ipage*4)
        imshow(themap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        colorbar()
        title('Fitted Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 4+ipage*4)
        imshow(flatmap-themap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        colorbar()
        title('Residuals Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        if (ipage == (nperpage-1)) or TESNum==TESmax:
            tight_layout()
            pdf.savefig()
            clf()
   
    
    
    
